You are working on a machine learning project where you have a dataset containing numerical and categorical features. You have identified that some of the features are highly correlated, and there are missing values in some of the columns. You want to build a pipeline that automates the feature engineering process and handles the missing values. 

Design a pipeline that includes the following steps:

- Use an automated feature selection method to identify the important features in the dataset.
- Create a numerical pipeline that includes the following steps:
-  Impute the missing values in the numerical columns using the mean of the column values.
- Scale the numerical columns using standardization.
- Create a categorical pipeline that includes the following steps:
- Impute the missing values in the categorical columns using the most frequent value of the column.
- One-hot encode the categorical columns.
- Combine the numerical and categorical pipelines using a ColumnTransformer.
- Use a Random Forest Classifier to build the final model.
- Evaluate the accuracy of the model on the test dataset.

Note : Your solution should include code snippets for each step of the pipeline, along with a brief explanation of each step. Additionally, provide an interpretation of the results and suggest possible improvements for the pipeline.

In [3]:
from sklearn.datasets import make_classification
import pandas as pd

# Generate a synthetic dataset
X, y = make_classification(n_samples=1000, n_features=10, n_informative=8, n_classes=2, random_state=42)

In [4]:


# Create a DataFrame
columns = [f"feature_{i}" for i in range(X.shape[1])]
df = pd.DataFrame(X, columns=columns)
df['category_feature'] = pd.Series(y).astype('category')


In [5]:
from sklearn.model_selection import train_test_split
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score



In [ ]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df.drop('category_feature', axis=1), df['category_feature'], test_size=0.2, random_state=42)


In [6]:

# Step 1: Automated Feature Selection
selector = SelectFromModel(RandomForestClassifier(random_state=42))
selector.fit(X_train, y_train)
X_train_selected = selector.transform(X_train)
X_test_selected = selector.transform(X_test)


In [ ]:
# Step 2: Numerical Pipeline
num_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='mean')),
    ('scaler', StandardScaler())
])

In [7]:



# Step 3: Categorical Pipeline
cat_pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('encoder', OneHotEncoder(handle_unknown='ignore'))
])

In [8]:

# Step 4: Combine Numerical and Categorical Pipelines
num_feature_indices = selector.get_support()
cat_feature_indices = ~num_feature_indices

preprocessor = ColumnTransformer([
    ('num', num_pipeline, num_feature_indices),
    ('cat', cat_pipeline, cat_feature_indices)
])

X_train_preprocessed = preprocessor.fit_transform(X_train)
X_test_preprocessed = preprocessor.transform(X_test)


In [9]:


# Step 5: Build Random Forest Classifier
clf = RandomForestClassifier(random_state=42)
clf.fit(X_train_preprocessed, y_train)

# Step 6: Evaluate Model
y_pred = clf.predict(X_test_preprocessed)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 0.765


Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its accuracy.

In [33]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import warnings 
warnings.filterwarnings("ignore")

In [34]:
# Load the Iris dataset
iris = load_iris()
X, y = iris.data, iris.target

In [35]:
# Split the dataset into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [36]:
# Step 1: Create individual classifiers
rf_classifier = RandomForestClassifier(random_state=42)
lr_classifier = LogisticRegression(random_state=42)


In [37]:
# Step 2: Create a Voting Classifier combining the individual classifiers
voting_classifier = VotingClassifier(
    estimators=[('rf', rf_classifier), ('lr', lr_classifier)],
    voting='soft'  # 'soft' for weighted average probabilities
)

In [38]:
# Step 3: Train the pipeline
voting_classifier.fit(X_train, y_train)

VotingClassifier(estimators=[('rf', RandomForestClassifier(random_state=42)),
                             ('lr', LogisticRegression(random_state=42))],
                 voting='soft')

In [39]:



# Step 4: Evaluate the model
y_pred = voting_classifier.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)


Accuracy: 1.0
